Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
# See cleaning and exploration in LS_DS_231

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/My Drive/Lambda/Unit 2/Project Data/shot_logs.csv')
df.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


#For the evaluation metric you chose, what score would you get just by guessing

In [5]:
#For the evaluation metric you chose, what score would you get just by guessing?
#54.8%
target = 'SHOT_RESULT'
y = df[target]
y.value_counts(normalize=True)

missed    0.547861
made      0.452139
Name: SHOT_RESULT, dtype: float64

#Can you make a fast, first model that beats guessing?

In [22]:
#Drop features that might cause leakage (FGM, PTS)
df = df.drop(['FGM', 'PTS'], axis=1)
df.head()


,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,brian roberts,203148


In [23]:
#Can you make a fast, first model that beats guessing?
import numpy as np
from sklearn.model_selection import train_test_split

train = df

# Split df into into train & test sets
train, test = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['SHOT_RESULT'], random_state=42)
train.shape, test.shape

((102455, 19), (25614, 19))

In [24]:
#Split df into train and validation sets
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['SHOT_RESULT'], random_state=42)
train.shape, val.shape

((81964, 19), (20491, 19))

In [0]:
# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [27]:
#install category encoders
!pip install category_encoders

In [29]:
#Fit a model using a pipeline

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.one_hot.OneHotEncoder(),
    SimpleImputer(),
    RandomForestClassifier(n_estimators=100, n_jobs= -1, random_state=42)
)

pipeline.fit(X_train, y_train)
print(f'Valdiation accuracy: {pipeline.score(X_val, y_val)}')

Valdiation accuracy: 0.605046117807818


#Data Exploration/Cleaning continued

In [30]:
df.describe()

,GAME_ID,FINAL_MARGIN,SHOT_NUMBER,PERIOD,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_id
count,1.280690e+05,128069.000000,128069.000000,128069.000000,122502.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000
mean,2.140045e+07,0.208723,6.506899,2.469427,12.453344,2.023355,2.765901,13.571504,2.264670,159038.487284,4.123015,157238.251247
std,2.578773e+02,13.233267,4.713260,1.139919,5.763265,3.477760,3.043682,8.888964,0.441159,78791.172947,2.756446,79362.389336
min,2.140000e+07,-53.000000,1.000000,1.000000,0.000000,0.000000,-163.600000,0.000000,2.000000,708.000000,0.000000,708.000000
25%,2.140023e+07,-8.000000,3.000000,1.000000,8.200000,0.000000,0.900000,4.700000,2.000000,101249.000000,2.300000,101162.000000
50%,2.140045e+07,1.000000,5.000000,2.000000,12.300000,1.000000,1.600000,13.700000,2.000000,201949.000000,3.700000,201939.000000
75%,2.140067e+07,9.000000,9.000000,3.000000,16.675000,2.000000,3.700000,22.500000,3.000000,203079.000000,5.300000,202704.000000
max,2.140091e+07,53.000000,38.000000,7.000000,24.000000,32.000000,24.900000,47.200000,3.000000,530027.000000,53.200000,204060.000000


In [31]:
df.dtypes

GAME_ID                         int64
MATCHUP                        object
LOCATION                       object
W                              object
FINAL_MARGIN                    int64
SHOT_NUMBER                     int64
PERIOD                          int64
GAME_CLOCK                     object
SHOT_CLOCK                    float64
DRIBBLES                        int64
TOUCH_TIME                    float64
SHOT_DIST                     float64
PTS_TYPE                        int64
SHOT_RESULT                    object
CLOSEST_DEFENDER               object
CLOSEST_DEFENDER_PLAYER_ID      int64
CLOSE_DEF_DIST                float64
player_name                    object
player_id                       int64
dtype: object

#Features to convert to ints/floats
GAME_CLOCK

#Create minutes and seconds int features from the 'Game clock' feature

In [32]:
df['MINUTES'] = 0
df['SECONDS'] = 0
df.head(1)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,0,0


In [34]:
counter = 0 

for x in df['GAME_CLOCK']:
  time = x.split(':')
  minute = time[0]
  second = time[1]
  minute = int(minute)
  second = int(second)
  df.loc[counter, 'MINUTES'] = minute
  df.loc[counter, 'SECONDS'] = second
  test = df.loc[counter, 'MINUTES']
  test2 = df.loc[counter, 'SECONDS']
  #create a variable to print on every 10000th iteration
  show = counter%1000
  if(show==0):
    print(df.loc[counter, 'MINUTES'], df.loc[counter, 'SECONDS'])
  counter = counter + 1

1 9
4 23
3 38
1 43
2 31
8 27
4 13
11 18
11 20
3 23
0 33
7 56
5 33
4 18
10 2
5 10
8 30
10 41
7 12
10 24
10 38
0 29
2 14
6 41
2 4
9 40
0 43
0 48
10 25
3 16
11 29
11 50
0 59
11 18
6 18
0 37
4 30
0 59
6 25
4 39
8 37
4 17
7 51
0 38
9 47
1 58
5 35
8 54
3 36
4 11
3 39
3 1
0 54
4 5
6 43
9 22
0 17
8 13
6 33
11 49
11 3
6 43
11 28
3 46
11 34
4 39
3 58
11 41
8 51
3 32
8 21
7 1
11 28
11 9
7 36
0 3
1 16
9 43
6 12
6 11
8 6
1 45
6 18
4 54
9 40
4 3
7 24
9 35
10 53
11 29
11 3
4 24
2 13
6 42
10 33
5 50
9 22
2 8
11 7
5 19
3 51
10 30
11 20
8 45
9 27
8 8
10 19
9 55
1 32
9 31
7 31
4 55
2 31
0 39
2 14
0 41
2 4
1 7
3 38
2 19
0 48
11 37
5 10
5 48
4 45
11 44
10 13
11 2
7 4


In [35]:
df.head(1)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9


#Other features to engineer: 
- seconds left in game
- points scored by player in game
- shooting percentage in game
- clutch factor: final margin, period, game clock, shot clock, 

#Seconds left in game

In [36]:
#create the feature
df['SECONDS_IN_GAME'] = 0
df.head(1)


,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9,0


In [39]:
index = 0

for x in range(5):
  period = df.loc[index, 'PERIOD']
  per_secs = (4-period)*720
  minutes = df.loc[index, 'MINUTES']
  min_secs = minutes*60
  seconds = df.loc[index, 'SECONDS']

  df.loc[index, 'SECONDS_IN_GAME'] = per_secs + min_secs + seconds
  index = index + 1

df.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9,2229
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,brian roberts,203148,0,14,2174
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,brian roberts,203148,0,0,2160
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,brian roberts,203148,11,47,2147
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,brian roberts,203148,10,34,2074


In [41]:
index = 0

for x in df['PERIOD']:
  period = df.loc[index, 'PERIOD']
  per_secs = (4-period)*720
  minutes = df.loc[index, 'MINUTES']
  min_secs = minutes*60
  seconds = df.loc[index, 'SECONDS']

  df.loc[index, 'SECONDS_IN_GAME'] = per_secs + min_secs + seconds
  index = index + 1

df.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,1.9,7.7,2,made,"Anderson, Alan",101187,1.3,brian roberts,203148,1,9,2229
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,0.8,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,brian roberts,203148,0,14,2174
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,2.7,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,brian roberts,203148,0,0,2160
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,1.9,17.2,2,missed,"Brown, Markel",203900,3.4,brian roberts,203148,11,47,2147
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,2.7,3.7,2,missed,"Young, Thaddeus",201152,1.1,brian roberts,203148,10,34,2074


In [42]:
condition = df['PERIOD'] == 5
df[condition].head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME
146,21400600,"JAN 17, 2015 - CHA vs. IND",H,W,9,12,5,2:22,9.7,14,12.7,8.1,2,missed,"Rudez, Damjan",204014,0.9,brian roberts,203148,2,22,-578
190,21400459,"DEC 29, 2014 - CHA vs. MIL",H,L,-10,6,5,4:00,13.7,3,2.3,6.4,2,missed,"Mayo, O.J.",201564,1.8,brian roberts,203148,4,0,-480
191,21400459,"DEC 29, 2014 - CHA vs. MIL",H,L,-10,7,5,1:17,21.1,0,0.8,24.7,3,missed,"Mayo, O.J.",201564,0.0,brian roberts,203148,1,17,-643
403,21400600,"JAN 17, 2015 - CHA vs. IND",H,W,9,6,5,4:06,12.9,0,0.8,5.6,2,missed,"Hibbert, Roy",201579,2.8,bismack biyombo,202687,4,6,-474
404,21400600,"JAN 17, 2015 - CHA vs. IND",H,W,9,7,5,3:10,24.0,0,0.0,3.4,2,made,"Hibbert, Roy",201579,2.4,bismack biyombo,202687,3,10,-530


In [43]:
#account for overtime periods
index = 0 

for x in df['PERIOD']:
  minutes = df.loc[index, 'MINUTES']
  min_secs = minutes*60
  seconds = df.loc[index, 'SECONDS']
  if x>4:
    df.loc[index, 'SECONDS_IN_GAME'] = min_secs + seconds
    print('Overtime')
  else:
    print('No Overtime')
  
  index = index + 1

df[condition].head()

Streaming output truncated to the last 5000 lines.
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
No Overtime
N

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,player_name,player_id,MINUTES,SECONDS,SECONDS_IN_GAME
146,21400600,"JAN 17, 2015 - CHA vs. IND",H,W,9,12,5,2:22,9.7,14,12.7,8.1,2,missed,"Rudez, Damjan",204014,0.9,brian roberts,203148,2,22,142
190,21400459,"DEC 29, 2014 - CHA vs. MIL",H,L,-10,6,5,4:00,13.7,3,2.3,6.4,2,missed,"Mayo, O.J.",201564,1.8,brian roberts,203148,4,0,240
191,21400459,"DEC 29, 2014 - CHA vs. MIL",H,L,-10,7,5,1:17,21.1,0,0.8,24.7,3,missed,"Mayo, O.J.",201564,0.0,brian roberts,203148,1,17,77
403,21400600,"JAN 17, 2015 - CHA vs. IND",H,W,9,6,5,4:06,12.9,0,0.8,5.6,2,missed,"Hibbert, Roy",201579,2.8,bismack biyombo,202687,4,6,246
404,21400600,"JAN 17, 2015 - CHA vs. IND",H,W,9,7,5,3:10,24.0,0,0.0,3.4,2,made,"Hibbert, Roy",201579,2.4,bismack biyombo,202687,3,10,190


#make a second model

In [44]:
train = df

# Split df into into train & test sets
train, test = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['SHOT_RESULT'], random_state=42)
train.shape, test.shape

((102455, 22), (25614, 22))

In [45]:
# Split df into into train & test sets
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['SHOT_RESULT'], random_state=42)
train.shape, val.shape

((81964, 22), (20491, 22))

In [0]:
# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [0]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [48]:
pipeline.fit(X_train, y_train)
print(f'Valdiation accuracy: {pipeline.score(X_val, y_val)}')

Valdiation accuracy: 0.6085598555463374
